In [ ]:
!mkdir MOAT3
%cd MOAT3

/content/MOAT3


In [ ]:
!git clone https://github.com/google-research/deeplab2

Cloning into 'deeplab2'...
remote: Enumerating objects: 2252, done.
remote: Counting objects: 100% (1008/1008), done.
remote: Compressing objects: 100% (220/220), done.
remote: Total 2252 (delta 847), reused 788 (delta 788), pack-reused 1244 (from 1)
Receiving objects: 100% (2252/2252), 8.42 MiB | 17.48 MiB/s, done.
Resolving deltas: 100% (1598/1598), done.


### Installation

In [6]:
!pip install tensorflow==2.15.0 keras==2.15.0 --force-reinstall

  Using cached tensorflow-2.15.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (4.4 kB)
  Using cached keras-2.15.0-py3-none-any.whl.metadata (2.4 kB)
  Using cached absl_py-2.3.1-py3-none-any.whl.metadata (3.3 kB)
  Using cached astunparse-1.6.3-py2.py3-none-any.whl.metadata (4.4 kB)
  Using cached flatbuffers-25.2.10-py2.py3-none-any.whl.metadata (875 bytes)
  Using cached gast-0.6.0-py3-none-any.whl.metadata (1.3 kB)
  Using cached google_pasta-0.2.0-py3-none-any.whl.metadata (814 bytes)
  Using cached h5py-3.14.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (2.7 kB)
  Using cached libclang-18.1.1-py2.py3-none-manylinux2010_x86_64.whl.metadata (5.2 kB)
  Using cached ml_dtypes-0.2.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (20 kB)
  Using cached numpy-1.26.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (61 kB)
  Using cached opt_einsum-3.4.0-py3-none-any.whl.metadata (6.3 kB)
  Using cache

In [2]:
!sudo apt-get install protobuf-compiler

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
protobuf-compiler is already the newest version (3.12.4-1ubuntu7.22.04.4).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.


In [3]:
!pip install protobuf==3.20.3

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.1/162.1 kB 12.8 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 4.25.8
    Uninstalling protobuf-4.25.8:
      Successfully uninstalled protobuf-4.25.8
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tf-keras 2.19.0 requires tensorflow<2.20,>=2.19, but you have tensorflow 2.15.0 which is incompatible.
tensorflow-text 2.19.0 requires tensorflow<2.20,>=2.19.0, but you have tensorflow 2.15.0 which is incompatible.
tensorflow-decision-forests 1.12.0 requires tensorflow==2.19.0, but you have tensorflow 2.15.0 which is incompatible.
grpcio-status 1.71.2 requires protobuf<6.0dev,>=5.26.1, but you have protobuf 3.20.3 which is incompatible.
keras-hub 0.21.1 requires keras>=3.5, but you have keras 2.15.0 which is incompatible.
ydf 0.13.0 requires protobuf<7.0.0,>=5.29.1, but you 

In [4]:
!pip install pillow
!pip install matplotlib
!pip install cython

In [5]:
%cd /content/MOAT3
!git clone https://github.com/tensorflow/models.git
!git clone https://github.com/cocodataset/cocoapi.git
%cd /content/MOAT3/cocoapi/PythonAPI
!make

/content/MOAT3
Cloning into 'models'...
remote: Enumerating objects: 102233, done.
remote: Counting objects: 100% (302/302), done.
remote: Compressing objects: 100% (168/168), done.
remote: Total 102233 (delta 200), reused 135 (delta 134), pack-reused 101931 (from 2)
Receiving objects: 100% (102233/102233), 644.65 MiB | 15.66 MiB/s, done.
Resolving deltas: 100% (72859/72859), done.
Cloning into 'cocoapi'...
remote: Enumerating objects: 975, done.
remote: Total 975 (delta 0), reused 0 (delta 0), pack-reused 975 (from 1)
Receiving objects: 100% (975/975), 11.72 MiB | 8.18 MiB/s, done.
Resolving deltas: 100% (576/576), done.
/content/MOAT3/cocoapi/PythonAPI
python setup.py build_ext --inplace
running build_ext
Compiling pycocotools/_mask.pyx because it changed.
[1/1] Cythonizing pycocotools/_mask.pyx
/usr/local/lib/python3.11/dist-packages/Cython/Compiler/Main.py:381: FutureWarning: Cython directive 'language_level' not set, using '3str' for now (Py3). This has changed from earlier releas

In [6]:
!echo $PYTHONPATH

/env/python


In [7]:
%cd /content/MOAT3
!export PYTHONPATH=$PYTHONPATH:`pwd`:`pwd`/models:`pwd`/cocoapi/PythonAPI

/content/MOAT3


In [8]:
%cd /content/MOAT3
!protoc deeplab2/*.proto --python_out=.

/content/MOAT3


In [9]:
!pip install numpy==1.26.4 --force-reinstall

  Using cached numpy-1.26.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (61 kB)
Using cached numpy-1.26.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (18.3 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tf-keras 2.19.0 requires tensorflow<2.20,>=2.19, but you have tensorflow 2.15.0 which is incompatible.
tensorflow-text 2.19.0 requires tensorflow<2.20,>=2.19.0, but you have tensorflow 2.15.0 which is incompatible.
opencv-python-headless 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.
tensorflow-decision-forests 1.12.0 requires tensorflow==2.19.0, but you have tensorflow 2.15.0 which is incompatible.
keras-hub

In [20]:
import os
os.environ['PYTHONPATH'] = (
    os.environ.get('PYTHONPATH', '') +
    f":{os.getcwd()}:{os.getcwd()}/models:{os.getcwd()}/cocoapi/PythonAPI"
)

In [11]:
!ls /usr/local/cuda/lib64/libcudart.so

/usr/local/cuda/lib64/libcudart.so


In [17]:
%%bash
set -x  # bật chế độ debug để in toàn bộ lệnh

cd /content/MOAT3

# Lấy cờ compile/link của TensorFlow
TF_CFLAGS=($(python3 -c 'import tensorflow as tf; print(" ".join(tf.sysconfig.get_compile_flags()))'))
TF_LFLAGS=($(python3 -c 'import tensorflow as tf; print(" ".join(tf.sysconfig.get_link_flags()))'))

OP_NAME="deeplab2/tensorflow_ops/kernels/merge_semantic_and_instance_maps_op"

# Bước 1: Compile CUDA kernel
nvcc -std=c++14 -c -o ${OP_NAME}_kernel.cu.o ${OP_NAME}_kernel.cu.cc \
${TF_CFLAGS[@]} -D GOOGLE_CUDA=1 -x cu -Xcompiler -fPIC --expt-relaxed-constexpr

# Bước 2: Link thành file .so
g++ -std=c++14 -shared -o ${OP_NAME}.so ${OP_NAME}.cc ${OP_NAME}_kernel.cc \
${OP_NAME}_kernel.cu.o ${TF_CFLAGS[@]} -fPIC -L/usr/local/cuda/lib64 -lcudart ${TF_LFLAGS[@]}

# Kiểm tra file .so có tồn tại không
ls -lh ${OP_NAME}.so


-rwxr-xr-x 1 root root 537K Aug 12 13:44 deeplab2/tensorflow_ops/kernels/merge_semantic_and_instance_maps_op.so


+ cd /content/MOAT3
+ TF_CFLAGS=($(python3 -c 'import tensorflow as tf; print(" ".join(tf.sysconfig.get_compile_flags()))'))
++ python3 -c 'import tensorflow as tf; print(" ".join(tf.sysconfig.get_compile_flags()))'
2025-08-12 13:44:02.223232: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-08-12 13:44:02.223306: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-08-12 13:44:02.225250: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-08-12 13:44:02.236273: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instruction

In [18]:
!ls /content/MOAT3/deeplab2/tensorflow_ops/kernels

merge_semantic_and_instance_maps_op.cc
merge_semantic_and_instance_maps_op_kernel.cc
merge_semantic_and_instance_maps_op_kernel.cu.cc
merge_semantic_and_instance_maps_op_kernel.cu.o
merge_semantic_and_instance_maps_op_kernel.h
merge_semantic_and_instance_maps_op.so


In [21]:
%cd /content/MOAT3
!python deeplab2/tensorflow_ops/python/kernel_tests/merge_semantic_and_instance_maps_op_test.py

/content/MOAT3
2025-08-12 13:46:58.633969: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-08-12 13:46:58.634023: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-08-12 13:46:58.635334: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-08-12 13:46:58.644275: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-08-12 13:47:00.159210: W tensorfl

#Test setup

In [ ]:
%cd /content/MOAT3
!python deeplab2/model/deeplab_test.py

/content/MOAT3
2025-08-11 13:35:03.292688: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-08-11 13:35:03.340904: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-08-11 13:35:03.341416: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-08-11 13:35:04.685145: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2025-08-11 13:35:07.192779: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torv

In [ ]:
%cd /content/MOAT3
!python deeplab2/trainer/evaluator_test.py

/content/MOAT3
2025-08-11 13:56:47.235418: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-08-11 13:56:47.236647: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-08-11 13:56:47.238284: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-08-11 13:56:47.248442: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-08-11 13:56:49.730841: W 

### Prepare dataset

In [22]:
%cd /content/MOAT3/deeplab2/data
!mkdir ade
%cd ade
!curl -L -o ADEChallengeData2016.zip\
  https://www.kaggle.com/api/v1/datasets/download/awsaf49/ade20k-dataset

/content/MOAT3/deeplab2/data
/content/MOAT3/deeplab2/data/ade
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 1124M  100 1124M    0     0  21.2M      0  0:00:52  0:00:52 --:--:-- 20.9M


In [23]:
%cd /content/MOAT3/deeplab2/data/ade
!unzip ADEChallengeData2016.zip

Kết quả truyền trực tuyến bị cắt bớt đến 5000 dòng cuối.
  inflating: ADEChallengeData2016/images/validation/ADE_val_00000357.jpg  
  inflating: ADEChallengeData2016/images/validation/ADE_val_00000358.jpg  
  inflating: ADEChallengeData2016/images/validation/ADE_val_00000359.jpg  
  inflating: ADEChallengeData2016/images/validation/ADE_val_00000360.jpg  
  inflating: ADEChallengeData2016/images/validation/ADE_val_00000361.jpg  
  inflating: ADEChallengeData2016/images/validation/ADE_val_00000362.jpg  
  inflating: ADEChallengeData2016/images/validation/ADE_val_00000363.jpg  
  inflating: ADEChallengeData2016/images/validation/ADE_val_00000364.jpg  
  inflating: ADEChallengeData2016/images/validation/ADE_val_00000365.jpg  
  inflating: ADEChallengeData2016/images/validation/ADE_val_00000366.jpg  
  inflating: ADEChallengeData2016/images/validation/ADE_val_00000367.jpg  
  inflating: ADEChallengeData2016/images/validation/ADE_val_00000368.jpg  
  inflating: ADEChallengeData2016/images/va

In [28]:
%%writefile /content/MOAT3/deeplab2/data/build_ade20k_data_semantic.py

# coding=utf-8
# Copyright 2025
#
# Licensed under the Apache License, Version 2.0

"""Converts ADE20K semantic segmentation data to sharded TFRecord format."""

import math
import os
from typing import Sequence

from absl import app
from absl import flags
from absl import logging
import tensorflow as tf

from deeplab2.data import data_utils

FLAGS = flags.FLAGS

# Đường dẫn ADE20K root (thư mục ADEChallengeData2016)
_ADE20K_ROOT = flags.DEFINE_string(
    'ade20k_root', None,
    'ADE20K dataset root folder containing images/ and annotations/ subfolders.')

_OUTPUT_DIR = flags.DEFINE_string(
    'output_dir', None,
    'Path to save converted TFRecord of TensorFlow examples.')

_NUM_SHARDS = flags.DEFINE_integer(
    'num_shards', 100,
    'Number of shards for the output TFRecord files.')

# Cấu trúc thư mục
_FOLDERS_MAP = {
    'train': {
        'image': 'images/training',
        'label': 'annotations/training',
    },
    'val': {
        'image': 'images/validation',
        'label': 'annotations/validation',
    },
}

# Định dạng file
_DATA_FORMAT_MAP = {
    'image': 'jpg',
    'label': 'png',
}


def _get_files(dataset_root: str, dataset_split: str, data_type: str):
    """Trả về danh sách file ảnh hoặc label."""
    pattern = '*.%s' % _DATA_FORMAT_MAP[data_type]
    search_files = os.path.join(
        dataset_root, _FOLDERS_MAP[dataset_split][data_type], pattern)
    files = tf.io.gfile.glob(search_files)
    return sorted(files)


def _convert_dataset(ade20k_root: str, dataset_split: str, output_dir: str):
    """Convert ADE20K semantic split sang TFRecord."""
    image_files = _get_files(ade20k_root, dataset_split, 'image')
    label_files = _get_files(ade20k_root, dataset_split, 'label')

    if len(image_files) != len(label_files):
        raise ValueError(
            f"Mismatch between image count ({len(image_files)}) "
            f"and label count ({len(label_files)}) in split {dataset_split}"
        )

    num_images = len(image_files)
    num_per_shard = int(math.ceil(num_images / _NUM_SHARDS.value))

    for shard_id in range(_NUM_SHARDS.value):
        start_idx = shard_id * num_per_shard
        end_idx = min((shard_id + 1) * num_per_shard, num_images)
        if start_idx >= end_idx:
            break

        shard_filename = f"{dataset_split}-{shard_id:05d}-of-{_NUM_SHARDS.value:05d}.tfrecord"
        output_path = os.path.join(output_dir, shard_filename)

        with tf.io.TFRecordWriter(output_path) as writer:
            for i in range(start_idx, end_idx):
                # Đọc ảnh RGB
                with tf.io.gfile.GFile(image_files[i], 'rb') as f:
                    image_data = f.read()

                # Đọc label (mask semantic)
                with tf.io.gfile.GFile(label_files[i], 'rb') as f:
                    label_data = f.read()

                # Tên file gốc
                filename = os.path.basename(image_files[i])
                file_prefix = os.path.splitext(filename)[0]

                # Tạo tf.Example
                example = data_utils.create_tfexample(
                    image_data=image_data,
                    image_format='jpeg',
                    filename=file_prefix,
                    label_data=label_data,
                    label_format='png',
                    # check_is_rgb=True
                )
                writer.write(example.SerializeToString())

        logging.info('Wrote %s', output_path)


def main(unused_argv: Sequence[str]):
    tf.io.gfile.makedirs(_OUTPUT_DIR.value)
    for split in ('train', 'val'):
        logging.info('Processing %s split...', split)
        _convert_dataset(_ADE20K_ROOT.value, split, _OUTPUT_DIR.value)
    logging.info('Done.')


if __name__ == '__main__':
    flags.mark_flags_as_required(['ade20k_root', 'output_dir'])
    app.run(main)


Overwriting /content/MOAT3/deeplab2/data/build_ade20k_data_semantic.py


In [29]:
%cd /content/MOAT3
!python deeplab2/data/build_ade20k_data_semantic.py \
  --ade20k_root=deeplab2/data/ade/ADEChallengeData2016 \
  --output_dir=deeplab2/data/ade/ADEChallengeData2016_tfrecord

/content/MOAT3
2025-08-12 14:54:07.795016: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-08-12 14:54:07.795077: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-08-12 14:54:07.796389: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-08-12 14:54:07.804083: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-08-12 14:54:09.055696: W tensorfl

### Download pretrained weights


In [31]:
%cd /content/MOAT3/deeplab2
!mkdir pretrained
%cd pretrained
!wget https://storage.googleapis.com/gresearch/tf-deeplab/checkpoint/moat3_imagenet_22k_and_1k_512_w_position_embedding.tar.gz


/content/MOAT3/deeplab2
/content/MOAT3/deeplab2/pretrained
--2025-08-12 15:04:48--  https://storage.googleapis.com/gresearch/tf-deeplab/checkpoint/moat3_imagenet_22k_and_1k_512_w_position_embedding.tar.gz
Resolving storage.googleapis.com (storage.googleapis.com)... 142.250.4.207, 74.125.24.207, 172.253.118.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.250.4.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 706272176 (674M) [application/x-tar]
Saving to: ‘moat3_imagenet_22k_and_1k_512_w_position_embedding.tar.gz’

moat3_imagenet_22k_ 100%[===================>] 673.55M  22.7MB/s    in 64s     

2025-08-12 15:05:52 (10.6 MB/s) - ‘moat3_imagenet_22k_and_1k_512_w_position_embedding.tar.gz’ saved [706272176/706272176]



In [30]:
!mkdir /content/MOAT3/deeplab2/configs/ade20k/moat3

In [32]:
%%writefile /content/MOAT3/deeplab2/configs/ade20k/moat3/moat3_large_os32_res1281.textproto
# proto-file: deeplab2/config.proto
# proto-message: ExperimentOptions
#
# kMaX-DeepLab with ConvNeXt-large backbone with output stride 32.
#
############### PLEASE READ THIS BEFORE USING THIS CONFIG ###############
# Before using this config, you need to update the following fields:
# - experiment_name: Use a unique experiment name for each experiment.
# - pretrained_weights: Update the path to the pretrained weights.
# - train_dataset_options.file_pattern: Update the path to the
#   training set. e.g., your_dataset/train*.tfrecord
# - eval_dataset_options.file_pattern: Update the path to the
#   validation set, e.g., your_dataset/eval*.tfrecord
#########################################################################
#
# References:
# For ConvNeXt, see
# - Zhuang Liu, et al. "A ConvNet for the 2020s." In CVPR, 2022.
# For kMaX-DeepLab, see
# - Qihang Yu, et al. "k-means Mask Transformer." In ECCV, 2022.

# Use a unique experiment_name for each experiment.
experiment_name: "moat3_finetune_512_22k_ade20k_semantic"
model_options {
  backbone {
    # kMaX meta is used with the prefix "kmax_".
    name: "moat3_finetune_512_22k"
    drop_path_keep_prob: 0.3
    # We use pretrained_weights instead of initial_checkpoint for loading
    # specifical weights for backbone (e.g., converted pytorch weights).
    pretrained_weights: pretrained/moat3_imagenet_22k_and_1k_512_w_position_embedding.tar.gz
  }
  decoder {
    feature_key: "feature_semantic"
    decoder_channels: 256
    aspp_channels: 256
    atrous_rates: 6
    atrous_rates: 12
    atrous_rates: 18
  }
  max_deeplab {
    pixel_space_head {
      output_channels: 128
      head_channels: 256
    }
    auxiliary_low_level {
      feature_key: "res3"
      channels_project: 64
    }
    auxiliary_low_level {
      feature_key: "res2"
      channels_project: 32
    }
    auxiliary_semantic_head {
      output_channels: 151
      head_channels: 256
    }
    instance_discrimination_sample_k: 4096
    instance_discrimination_sample_temperature: 1.5
    semantic_sample_k: 4096
    semantic_sample_temperature: 2.0
  }
}
trainer_options {
  save_checkpoints_steps: 1000
  save_summaries_steps: 100
  steps_per_loop: 100
  loss_options {
    semantic_loss {
      name: "softmax_cross_entropy"
      weight: 1.0
    }
    pq_style_loss {
      weight: 3.0
    }
    mask_id_cross_entropy_loss {
      weight: 0.3
    }
    instance_discrimination_loss {
      weight: 1.0
    }
  }
  solver_options {
    base_learning_rate: 0.0005
    training_number_of_steps: 100000
    warmup_steps: 5000
    backbone_learning_rate_multiplier: 0.1
    optimizer: "adamw"
    adamw_weight_decay: 0.05
  }
}
train_dataset_options {
  dataset: "ade20k_semantic"
  file_pattern: data/ade/ADEChallengeData2016_tfrecord/train*.tfrecord
  # Adjust the batch_size accordingly to better fit your GPU/TPU memory.
  # Also see Q1 in g3doc/faq.md.
  batch_size: 64
  crop_size: 1281
  crop_size: 1281
  min_resize_value: 1281
  max_resize_value: 1281
  augmentations {
    min_scale_factor: 0.5
    max_scale_factor: 2.0
    scale_factor_step_size: 0.1
    autoaugment_policy_name: "simple_classification_policy"
    panoptic_copy_paste {
      panoptic_copy_paste_scale: 0.5
      panoptic_copy_paste_thing_option: "all"
      panoptic_copy_paste_stuff_option: "random"
      panoptic_copy_paste_void_option: "none"
    }
  }
  increase_small_instance_weights: false
  small_instance_weight: 1.0
  thing_id_mask_annotations: true
  max_thing_id: 256
}
eval_dataset_options {
  dataset: "ade20k_semantic"
  # Update the path to validation set.
  file_pattern: data/ade/ADEChallengeData2016_tfrecord/val*.tfrecord
  batch_size: 1
  crop_size: 1281
  crop_size: 1281
  min_resize_value: 1281
  max_resize_value: 1281
  # Add options to make the evaluation loss comparable to the training loss.
  increase_small_instance_weights: false
  small_instance_weight: 1.0
  thing_id_mask_annotations: true
  max_thing_id: 256
}
evaluator_options {
  continuous_eval_timeout: -1
  thing_area_limit: 0
  stuff_area_limit: 0
  transformer_post_processing: "maskwise"
  pixel_confidence_threshold: 0.4
  maskwise_postprocessing {
    transformer_class_confidence_threshold_thing: 0.4
    transformer_class_confidence_threshold_stuff: 0.4
    overlapping_threshold: 0.8
    reorder_class_weight: 1.0
    reorder_mask_weight: 1.0
  }
  save_predictions: true
  save_raw_predictions: false
}


Writing /content/MOAT3/deeplab2/configs/ade20k/moat3/moat3_large_os32_res1281.textproto


In [37]:
!apt-get update -y
!apt-get install python3.9 python3.9-distutils python3.9-dev -y
!update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.9 1
!update-alternatives --config python3

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:6 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:7 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:9 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,270 kB]
Hit:10 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Get:11 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,927 kB]
Get:12 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:13 http://archive.ubuntu.com/ubuntu jammy-up

In [38]:
!wget https://bootstrap.pypa.io/get-pip.py
!python3 get-pip.py

--2025-08-12 15:43:38--  https://bootstrap.pypa.io/get-pip.py
Resolving bootstrap.pypa.io (bootstrap.pypa.io)... 151.101.0.175, 151.101.64.175, 151.101.128.175, ...
Connecting to bootstrap.pypa.io (bootstrap.pypa.io)|151.101.0.175|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2148718 (2.0M) [text/x-python]
Saving to: ‘get-pip.py’

get-pip.py          100%[===================>]   2.05M  --.-KB/s    in 0.01s   

2025-08-12 15:43:38 (191 MB/s) - ‘get-pip.py’ saved [2148718/2148718]

  Using cached setuptools-80.9.0-py3-none-any.whl.metadata (6.6 kB)
  Using cached wheel-0.45.1-py3-none-any.whl.metadata (2.3 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 62.3 MB/s  0:00:00
Using cached setuptools-80.9.0-py3-none-any.whl (1.2 MB)
Using cached wheel-0.45.1-py3-none-any.whl (72 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3/3 [pip]


In [39]:
!python3 --version

Python 3.9.23


In [40]:
!pip install tensorflow==2.10.1 tensorflow-gpu==2.10.1

  Using cached absl_py-2.3.1-py3-none-any.whl.metadata (3.3 kB)
  Using cached astunparse-1.6.3-py2.py3-none-any.whl.metadata (4.4 kB)
  Using cached flatbuffers-25.2.10-py2.py3-none-any.whl.metadata (875 bytes)
  Using cached google_pasta-0.2.0-py3-none-any.whl.metadata (814 bytes)
  Using cached libclang-18.1.1-py2.py3-none-manylinux2010_x86_64.whl.metadata (5.2 kB)
  Using cached opt_einsum-3.4.0-py3-none-any.whl.metadata (6.3 kB)
  Using cached packaging-25.0-py3-none-any.whl.metadata (3.3 kB)
  Using cached termcolor-3.1.0-py3-none-any.whl.metadata (6.4 kB)
  Using cached typing_extensions-4.14.1-py3-none-any.whl.metadata (3.0 kB)
  Using cached google_auth-2.40.3-py2.py3-none-any.whl.metadata (6.2 kB)
  Using cached requests-2.32.4-py3-none-any.whl.metadata (4.9 kB)
  Using cached werkzeug-3.1.3-py3-none-any.whl.metadata (3.7 kB)
  Using cached cachetools-5.5.2-py3-none-any.whl.metadata (5.4 kB)
  Using cached pyasn1_modules-0.4.2-py3-none-any.whl.metadata (3.5 kB)
  Using cached

In [43]:
!pip install numpy==1.26.4 --force-reinstall

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 66.4 MB/s  0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2


In [ ]:
%cd /content/MOAT3/deeplab2/
!python trainer/train.py \
    --config_file=configs/ade20k/moat3/moat3_large_os32_res1281.textproto \
    --mode=train_and_eval \
    --model_dir=moat3_ade_20k_workdirs \
    --num_gpus=1